# Numpy 배열 연산: 유니버설 함수
1. 루프는 느리다
2. UFuncs 소개
3. Numpy 유니버설 함수(UFuncs)
4. 고급 UFunc 기능
5. 집계: 최솟갑, 최대값, 그리고 그 사이의 모든 것
6. 예제: 미국 대통령의 평균 신장은 얼마일까?

- Numpy는 데이터 배열을 사용ㅇ하여 최적화된 연산을 위한 쉽고 유연한 인터페이스 제공
- Numpy 배열 연산을 빠르게 만드는 핵심은 벡터화 -> universal function 통해 구현

### 1. 루프는 느리다: universal function의 필요성
- 파이썬 기본 구현(CPython)에서 몇 가지 연산은 매우 느리게 수행
- 부분적으로는 파이썬이 동적인 인터프리터 언어이기 때문
- 타입이 유연하다 = 연산이 C나 Fortran에서처럼 효율적인 머신 코드로 컴파일 되기 어려움
- 파이썬은 작은 연산이 반복되는 상황에서 확연히 느리다

Syntax
- numpy.random.randint(low, high=None, size=None, dtype=int)

[Summary]
- low: The lower bound (inclusive) of the random integers.
- high: The upper bound (exclusive) of the random integers. If not provided, the range is [0, low).
- size: The shape of the output array. If not provided, a single integer is returned.
- dtype: The data type of the output array. Default is int.

In [3]:
import numpy as np

# generate a single random number btw 0(inclusive) and 10(exclusive)
random_int = np.random.randint(10)
print(random_int)

#generate a single random integer bewtween 5(inclusive) and 10 (exclusive)
random_int2 = np.random.randint(5,10)
print(random_int2)

# generate a 1D array containing 5 random integers btw 0(inclusive) and 10(exclusive)
random_array = np.random.randint(0,10,size=5)
print(random_array)

# generate a 2D array containing 3 rows and 3 columns of random integers btw 0(inclusive) and 10(exclusive)
random_array2 = np.random.randint(0,10,(3,3))
print(random_array2)

# speficy data type
# generate a 1D array containing 5 random integers between 0(inclusive) and 10(exclusive) with data type int64
random_array_dtype = np.random.randint(0,10,size=5, dtype='int64')
print(random_array_dtype)

2
9
[1 3 4 8 6]
[[5 8 3]
 [1 9 6]
 [9 3 6]]
[8 5 1 5 6]


In [4]:
# example. 값으로 이뤄진 배열이 있고 각각의 역수를 계산하려고 한다.

np.random.seed(0)
def compute_reciprocals(values):
    output = np.empty(len(values))
    for i in range(len(values)):
        output[i] = 1.0 / values[i]
    return output


values = np.random.randint(1, 10, size=5)
compute_reciprocals(values)

array([0.16666667, 1.        , 0.25      , 0.25      , 0.125     ])

In [5]:
# 이 코드는 큰 배열을 다룰 때 매우 느리다. 이유는 반복문을 사용하고 있기 때문이다.
# %timeit을 사용해 속도 측정

big_array = np.random.randint(1, 100, size = 1000000)
%timeit compute_reciprocals(big_array)

2.53 s ± 222 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


 병목은 연산 자체에 있는 것이 아니라 CPython이 루프의 사이클마다 수행해야 하는 타입 확인과 함수 디스패치에서 발생함
- CPython 사이클: 객체 타입 확인 > 해당 타입에 맞게 사용할 함수 동적 검색 > 이후 연산
- 컴파일된 코드로 작업했다면 코드 실행 전 타입을 알았을 것. 결괏값 계산 효율적이었을 것.

### 2. UFuncs 소개
NumPy는 여러 종류의 연산에 대해 정적 타입 체계를 가진 컴파일된 루틴에 편리한 인터페이스 제공
- 벡터화 연산: 배열에 연산을 수행해 각 요소에 적용함. 루프를 NumPy의 기저를 이루는 컴파일된 계층으로 밀어넣음으로써 훨씬 빠르게 실행되도록 설계됨 

In [6]:
print(compute_reciprocals(values))
print(1.0/values)

[0.16666667 1.         0.25       0.25       0.125     ]
[0.16666667 1.         0.25       0.25       0.125     ]


In [ ]:
# 파이썬 루프보다 이 실행코드가 수백 배 빠른 속도로 작업을 완료함
%timeit (1.0/big_array)

3.68 ms ± 132 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


- NumPy에서 벡터화 연산은 NumPy 배열의 값에 반복된 연산을 빠르게 수행하는 것을 주목적으로 하는 ufuncs를 통해 구현됨
- 유니버설 함수는 매우 유연: 스칼라<>배열, 배열<>배열, 다차원 배열..

In [15]:
# 스칼라<>배열
1.0/big_array

array([0.1       , 0.01190476, 0.04545455, ..., 0.01428571, 0.01098901,
       0.01149425])

In [16]:
# 배열<>배열
np.arange(5)/np.arange(1,6)


array([0.        , 0.5       , 0.66666667, 0.75      , 0.8       ])

1. True Division (/):
Always returns a floating-point result.
Preserves the fractional part of the division.

2. Floor Division (//):
Returns the largest integer less than or equal to the division result.
Discards the fractional part of the division.

In [14]:
# 다차원 배열
x = np.arange(9).reshape((3,3))
x**2

array([[ 0,  1,  4],
       [ 9, 16, 25],
       [36, 49, 64]])